### RME por causa de fallecimiento y CCAA
Se calcula el RME para cada causa de fallecimiento por CCAA, esto nos permite comparar las CCAA quitandole el efecto de los diferentes mixes de edad y sexo.

Para el cálculo de RME:
* Se calcula la **frecuencia esperada por causa de fallecimiento**: la frecuencia de fallecimiento por causa a nivel estatal para cada grupo de edad y sexo.
* Se calcula los **fallecimientos esperados por causa de cada CCAA**: se multiplica la frecuencia esperada calculada por la población de la CCAA para cada grupo de edad y sexo.
* Se calcula el **RME**:
$$
RME_{cit} = \frac{\sum_{edad, sexo} SINIESTROS^{OBS}_{edad, sexo, c, i , t}}{\sum_{edad, sexo} SINIESTROS^{EST}_{edad, sexo, c, i , t}}
$$
<p align="center">Siendo <strong>c</strong> la <i>causa de fallecimiento</i>, <strong>i</strong> la <i>CCAA</i> y <strong>t</strong> el <i>año</i>.</p>

El análisis se ha baso en el [Atlas Nacional de Mortalidad en España (ANDEES)](https://medea3.shinyapps.io/atlas_nacional/), que fue desarrollado por el grupo de investigación Bayensians de la Fundación FISABIO y la Dirección General de Salud Pública de la Generalitat Valenciana.

Fuentes:
* Defuenciones según causa de muerte: https://www.ine.es/jaxiT3/Tabla.htm?t=10803&L=0
* Población por comunidades, edad (grupos quinquenales), Españoles/Extranjeros, Sexo y Año: https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=02002.px&L=0


In [ ]:
import pandas as pd

### Diccionario para normalizar edades

In [8]:
pob_edad_dict ={'0-4 años':'De 0 a 14 años', 
                '5-9 años':'De 0 a 14 años', 
                '10-14 años':'De 0 a 14 años', 
                '15-19 años':'De 15 a 29 años',
                '20-24 años':'De 15 a 29 años', 
                '25-29 años':'De 15 a 29 años', 
                '30-34 años':'De 30 a 39 años', 
                '35-39 años':'De 30 a 39 años',
                '40-44 años':'De 40 a 44 años', 
                '45-49 años':'De 45 a 49 años', 
                '50-54 años':'De 50 a 54 años', 
                '55-59 años':'De 55 a 59 años', 
                '60-64 años':'De 60 a 64 años', 
                '65-69 años':'De 65 a 69 años', 
                '70-74 años':'De 70 a 74 años', 
                '75-79 años':'De 75 a 79 años', 
                '80-84 años':'De 80 a 84 años', 
                '85-89 años':'De 85 a 89 años', 
                '90-94 años':'De 90 a 94 años', 
                '95-99 años':'De 95 años y más', 
                '100 años y más':'De 95 años y más'}

In [9]:
dec_edad_dict = {'Menos de 1 año':'De 0 a 14 años','De 1 a 14 años':'De 0 a 14 años','95 y más años':'De 95 años y más'}

### Importamos poblaciones

In [43]:
df_pob = pd.read_csv('./input/ine_poblacion_x_ccaa.csv', sep=';', encoding='utf8', thousands='.', decimal=',')
df_pob.columns = ['nacional', 'ccaa', 'edad','esp_ext','sexo','periodo', 'total']
df_pob.head()

,nacional,ccaa,edad,esp_ext,sexo,periodo,total
0,Total Nacional,NaN,TOTAL EDADES,TOTAL,Ambos sexos,2022,47475420
1,Total Nacional,NaN,TOTAL EDADES,TOTAL,Ambos sexos,2021,47385107
2,Total Nacional,NaN,TOTAL EDADES,TOTAL,Ambos sexos,2020,47450795
3,Total Nacional,NaN,TOTAL EDADES,TOTAL,Ambos sexos,2019,47026208
4,Total Nacional,NaN,TOTAL EDADES,TOTAL,Ambos sexos,2018,46722980


In [44]:
df_pob['edad'] = df_pob.edad.apply(lambda x: x.strip() if pob_edad_dict.get(x) is None else pob_edad_dict.get(x))
df_pob['ccaa'] = df_pob.ccaa.fillna('Total Nacional')
df_pob= df_pob.groupby(['nacional', 'ccaa', 'edad', 'esp_ext', 'sexo', 'periodo']).sum().reset_index()

In [45]:
pob_sexo_edad = df_pob[(df_pob.ccaa=='Total Nacional') & (df_pob.sexo != 'Ambos sexos') & (df_pob.edad != 'TOTAL EDADES') & (df_pob.esp_ext=='TOTAL') ].groupby(['sexo','edad','periodo'])[['total']].sum()

### Importamos causas de fallecimiento

In [46]:
df_causas = pd.read_csv('./input/ine_defunciones_x_ccaa_2.csv', sep=';', encoding='utf8', thousands='.', decimal=',')
df_causas.columns = ['causas','sexo','edad','ccaa','periodo','total']
df_causas.head()

,causas,sexo,edad,ccaa,periodo,total
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,Nacional,2022,464417
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,Nacional,2021,450744
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,Nacional,2020,493776
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,Nacional,2019,418703
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,Nacional,2018,427721


In [47]:
df_causas['edad'] = df_causas.edad.apply(lambda x: x.strip() if dec_edad_dict.get(x) is None else dec_edad_dict.get(x))
df_causas = df_causas.groupby(['causas', 'sexo', 'edad', 'ccaa', 'periodo'])['total'].sum().reset_index()

In [48]:
causas_del = ['001-008  I.Enfermedades infecciosas y parasitarias','001-102  I-XXII.Todas las causas','001-102 I-XXII.Todas las causas']

In [52]:
fall_xcausa_sexo_edad = df_causas[(df_causas.ccaa != 'Nacional') & (df_causas.sexo != 'Total') & (df_causas.edad != 'Todas las edades') & (df_causas.causas.isin(causas_del)==False)].groupby(['causas','sexo','edad','periodo'])[['total']].sum()
fall_xcausa_sexo_edad.head()

total
causas                                     sexo    edad           periodo       
001  Enfermedades infecciosas intestinales Hombres De 0 a 14 años 1980        62
                                                                  1981        45
                                                                  1982        37
                                                                  1983        32
                                                                  1984        23

In [54]:
freq_esperada = fall_xcausa_sexo_edad\
    .reset_index()\
    .merge(pob_sexo_edad.reset_index().rename(columns={'total':'poblacion','edad2':'edad'}),
           on=['sexo','edad','periodo'])\
    .assign(freq_tot_esp = lambda x: x.total / x.poblacion)
freq_esperada.head()

,causas,sexo,edad,periodo,total,poblacion,freq_tot_esp
0,001 Enfermedades infecciosas intestinales,Hombres,De 0 a 14 años,1998,5,3098799,1.613528e-06
1,001 Enfermedades infecciosas intestinales,Hombres,De 0 a 14 años,1999,1,3052063,3.276472e-07
2,001 Enfermedades infecciosas intestinales,Hombres,De 0 a 14 años,2000,4,3008304,1.329653e-06
3,001 Enfermedades infecciosas intestinales,Hombres,De 0 a 14 años,2001,5,3003271,1.664851e-06
4,001 Enfermedades infecciosas intestinales,Hombres,De 0 a 14 años,2002,2,3034646,6.590555e-07


In [60]:
df_pob_xCom = df_pob[(df_pob.ccaa.isnull()==False) & (df_pob.sexo != 'Ambos sexos') & (df_pob.edad != 'TOTAL EDADES') ].copy()
df_pob_xCom\
    .groupby(['ccaa','edad','esp_ext','sexo','periodo'])[['total']]\
    .sum().head()

total
ccaa         edad           esp_ext   sexo    periodo        
01 Andalucía De 0 a 14 años Españoles Hombres 1998     674484
                                              1999     661273
                                              2000     644355
                                              2001     634222
                                              2002     627365

In [61]:
df_pob_xCom = df_pob[(df_pob.ccaa.isnull()==False) & (df_pob.sexo != 'Ambos sexos') & (df_pob.edad != 'TOTAL EDADES') & (df_pob.esp_ext == 'TOTAL') ].copy()
df_pob_xCom = df_pob_xCom\
    .groupby(['ccaa','edad','sexo','periodo'])[['total']]\
    .sum()\
    .reset_index()\
    .merge(freq_esperada[['causas','sexo','edad','freq_tot_esp','periodo']], how='left', on=['sexo','edad','periodo'])\
    .assign(fall_esperados = lambda x: x.freq_tot_esp * x.total)

In [63]:
df_pob_xCom.head()
#df_causas[(df_causas.ccaa=='01 Andalucía') & (df_causas.edad=='De 0 a 14 años') & (df_causas.sexo=='Hombres') & (df_causas.periodo==1998) & (df_causas.causas=='001  Enfermedades infecciosas intestinales')]
#  & (df_causas.causas=='001  Enfermedades infecciosas intestinales')

,ccaa,edad,sexo,periodo,total,causas,freq_tot_esp,fall_esperados
0,01 Andalucía,De 0 a 14 años,Hombres,1998,679553,001 Enfermedades infecciosas intestinales,1.613528e-06,1.096478
1,01 Andalucía,De 0 a 14 años,Hombres,1998,679553,002 Tuberculosis y sus efectos tardíos,0.000000e+00,0.000000
2,01 Andalucía,De 0 a 14 años,Hombres,1998,679553,003 Enfermedad meningocócica,7.422230e-06,5.043799
3,01 Andalucía,De 0 a 14 años,Hombres,1998,679553,004 Septicemia,5.163291e-06,3.508730
4,01 Andalucía,De 0 a 14 años,Hombres,1998,679553,005 Hepatitis vírica,3.227057e-07,0.219296


In [64]:
def grup_periodo(e):
    if e<=1997:
        return 0 
    elif e<=2000:
        return 2000
    elif e<=2005:
        return 2005
    elif e<=2010:
        return 2010
    elif e<=2015: 
        return 2015
    elif e<=2020:
        return 2020
    else:
        return 2022

In [72]:
fall_obs

,ccaa,sexo,group_periodo,causas,total
0,01 Andalucía,Hombres,2000,001 Enfermedades infecciosas intestinales,63
1,01 Andalucía,Hombres,2000,001-008 I.Enfermedades infecciosas y parasita...,2293
2,01 Andalucía,Hombres,2000,001-102 I-XXII.Todas las causas,96858
3,01 Andalucía,Hombres,2000,002 Tuberculosis y sus efectos tardíos,222
4,01 Andalucía,Hombres,2000,003 Enfermedad meningocócica,34
...,...,...,...,...,...
45355,Nacional,Total,2022,098 Suicidio y lesiones autoinfligidas,8230
45356,Nacional,Total,2022,099 Agresiones (homicidio),588
45357,Nacional,Total,2022,100 Eventos de intención no determinada,140
45358,Nacional,Total,2022,101 Complicaciones de la atención médica y qu...,322


In [73]:
fall_obs = df_causas[(df_causas.edad != 'Todas las edades') & (df_causas.periodo>1997)]\
    .assign(group_periodo = lambda x:x.periodo.apply(grup_periodo))\
    .groupby(['ccaa','sexo','group_periodo','causas'])[['total']]\
    .sum().reset_index().rename(columns={'total':'fall_observados'})
    
fall_esp = df_pob_xCom\
    .assign(group_periodo = lambda x:x.periodo.apply(grup_periodo))\
    .groupby(['ccaa','sexo','group_periodo','causas'])[['fall_esperados','total']]\
    .sum().reset_index()

dfFinal = fall_obs.merge(fall_esp, on=['ccaa','sexo','group_periodo','causas']).assign(RME = lambda x: x.fall_observados / x.fall_esperados)

In [75]:
dfFinal.to_csv('./RME_x_periodo_causa_comunidad_v2.csv', sep=';', encoding='utf8', index=False)

In [25]:
x.to_csv('./RME_x_periodo_causa_comunidad.csv', sep=';', encoding='utf8', index=False)

In [27]:
df_causas[(df_causas.ccaa.isnull() == False) & (df_causas.edad != 'Todas las edades') & (df_causas.sexo != 'Ambos sexos')  & (df_causas.causas.isin(causas_del)==False)]\
    .merge(freq_esperada, on=['causas','sexo','edad'])

,nacional,ccaa,causas,sexo,edad,total_x,total_y,poblacion,freq_esperada
0,Total Nacional,01 Andalucía,00A Covid-19 Virus identificado,Hombres,De 0 a 14 años,0.0,0.0,168965148,0.000000e+00
1,Total Nacional,01 Andalucía,00A Covid-19 Virus identificado,Hombres,De 0 a 14 años,0.0,0.0,168965148,0.000000e+00
2,Total Nacional,01 Andalucía,00A Covid-19 Virus identificado,Hombres,De 15 a 29 años,2.0,5.0,212605453,2.351774e-08
3,Total Nacional,01 Andalucía,00A Covid-19 Virus identificado,Hombres,De 30 a 39 años,0.0,4.0,181981437,2.198026e-08
4,Total Nacional,01 Andalucía,00A Covid-19 Virus identificado,Hombres,De 40 a 44 años,0.0,4.0,91260001,4.383081e-08
...,...,...,...,...,...,...,...,...,...
71739,Total Nacional,19 Melilla,102 Otras causas externas y sus efectos tardíos,Mujeres,De 75 a 79 años,0.0,2.0,45806000,4.366240e-08
71740,Total Nacional,19 Melilla,102 Otras causas externas y sus efectos tardíos,Mujeres,De 80 a 84 años,0.0,2.0,36187639,5.526749e-08
71741,Total Nacional,19 Melilla,102 Otras causas externas y sus efectos tardíos,Mujeres,De 85 a 89 años,0.0,3.0,23552988,1.273724e-07
71742,Total Nacional,19 Melilla,102 Otras causas externas y sus efectos tardíos,Mujeres,De 90 a 94 años,0.0,2.0,10283484,1.944866e-07


In [28]:
df_pob_xCom.fall_esperados.isnan()


AttributeError: 'Series' object has no attribute 'isnan'

In [35]:
(df_causas.nacional=='Total Nacional') & (df_causas.ccaa.isnull())

0          True
1          True
2          True
3          True
4          True
          ...  
128515    False
128516    False
128517    False
128518    False
128519    False
Length: 128520, dtype: bool